In [80]:
import numpy as np
import random
from matplotlib import pyplot as plt
import scipy as sp
import networkx as nx

### Uniform Spanning Tree

A spanning tree of a finite connected graph G is a connected subgraph of G containing every vertex and no cycles.  
(Ref: https://mathweb.ucsd.edu/~jschwein/LERW.pdf)

### Wilson's Algorithm

0. Create empty cells
1. Set the starting position
2. While it's not a spanning tree
    - Perform LERW in a network  
        1) Pick any random position  
        2) Perform a random walk  
        3) Check if the position is already visited  
            -- If yes, erase the loop and restart the walk
            -- If no, add to the trace
        4) Check if the current position is already in the trace

### Version 1.1

In [74]:
# Define a function to perform LERW

def LERW(start_position, goal, dim):
    
    visited_cell = []
    current_position = start_position
    visited_cell.append(current_position)
    while current_position != goal:
        neighbors = [(current_position[0],current_position[1]-1),(current_position[0],current_position[1]+1),
                     (current_position[0]-1,current_position[1]),(current_position[0]+1,current_position[1])] # (left,right,up,down)
        feasible_neighbors = [i for i in neighbors if (i[0] >= 0 and i[0] <= dim-1) and (i[1] >= 0 and i[1] <= dim-1)]
        next_position = random.choice(feasible_neighbors)
        # Check if there is a loop
        if next_position not in visited_cell:
            visited_cell.append(next_position)
        else:
            # reset path
            if visited_cell.index(next_position) == 0:
                visited_cell = [visited_cell[0]]
            else:
                visited_cell = visited_cell[:visited_cell.index(next_position)+1]

        current_position = next_position
    return start_position, goal, current_position, visited_cell

def cont_LERW(set_path, remaining_cell, dim):
    
    new_path = []
    new_start_position = random.choice(remaining_cell)
    current_position = new_start_position
    new_path.append(current_position)
    
    while current_position not in set_path:
        neighbors = [(current_position[0],current_position[1]-1),(current_position[0],current_position[1]+1),
                     (current_position[0]-1,current_position[1]),(current_position[0]+1,current_position[1])] # (left,right,up,down)
        feasible_neighbors = [i for i in neighbors if (i[0] >= 0 and i[0] <= dim-1) and (i[1] >= 0 and i[1] <= dim-1)]
        next_position = random.choice(feasible_neighbors)
        
        # Unprecedented path: add to the path
        if (next_position not in new_path) and (next_position not in set_path):
            new_path.append(next_position)
        
        # Adjoining path: add to the path and exit the loop
        if next_position in set_path:
            new_path.append(next_position)
            break
        
        # Loop detected: perform loop-erased RW
        if next_position in new_path:
            if new_path.index(next_position) == 0:
                new_path = [new_path[0]]
            else:
                new_path = new_path[:new_path.index(next_position)+1]
                
        # check if current_position is already in the path to goal.
        current_position = next_position
    return new_start_position, current_position, new_path

In [116]:
# Define a function sample_spanning_tree

def sample_spanning_tree(spMatrix):
    '''NOT DONE YET!
    REMAINING TASKS:
    1. Modify cont_LERW to correctly return the final result
    2. Return the sparse matrix representing network from the main function'''
    dim = spMatrix.getnnz()
    universe = [(i,j) for i in range(spMatrix.getnnz()) for j in range(spMatrix.getnnz())]
    start_position = random.choice(universe)
    universe.remove(start_position)
    goal = random.choice(universe)
    set_path = LERW(start_position, goal, dim)[3]
    
    # After obtaining the first path to the goal, generate the next RW until it's a spanning tree.
    remaining_cell = [i for i in universe if i not in set_path]
    while len(remaining_cell) > 0:
        add_path = cont_LERW(set_path, remaining_cell, dim)[2]
        set_path.extend(add_path)
        remaining_cell = [i for i in universe if i not in set_path]
    return set_path, remaining_cell, add_path

In [119]:
print(A)

  (0, 1)	1.0
  (1, 2)	1.0


In [128]:
sample_spanning_tree(A)

([(3, 1),
  (2, 1),
  (2, 2),
  (2, 3),
  (1, 3),
  (2, 3),
  (1, 0),
  (1, 1),
  (2, 1),
  (0, 2),
  (0, 3),
  (1, 3),
  (1, 2),
  (1, 3),
  (2, 0),
  (3, 0),
  (3, 1),
  (0, 0),
  (1, 0),
  (3, 3),
  (2, 3),
  (0, 1),
  (0, 0),
  (3, 2),
  (2, 2)],
 [],
 [(3, 2), (2, 2)])

In [111]:
A.getnnz()

2

### Version 1.2

In [529]:
G = nx.MultiDiGraph()
G.add_edge(0, 1)
G.add_edge(1, 0)
G.add_edge(2, 1)
G.add_edge(1, 2)
G.add_edge(0, 2)
G.add_edge(2, 0)
G.add_edge(3, 1)
G.add_edge(1, 3)
G.add_edge(2, 4)
G.add_edge(4, 2)
G.add_edge(3, 4)
G.add_edge(4, 3)
G.add_edge(2, 3)
G.add_edge(3, 2)
A = nx.to_scipy_sparse_matrix(G, nodelist=[0, 1, 2, 3, 4])

In [419]:
print(A)

  (0, 1)	1
  (0, 2)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (2, 0)	1
  (2, 1)	1
  (2, 3)	1
  (2, 4)	1
  (3, 1)	1
  (3, 2)	1
  (3, 4)	1
  (4, 2)	1


In [781]:
A = nx.to_scipy_sparse_matrix(G, nodelist=[0, 1, 2, 3, 4])
num_nodes = A.shape[0]
A = A.todense()
universe_node_list = list(range(num_nodes))
unused_node_list = list(range(num_nodes))
selected_node = [False for node in range(num_nodes)]
output = [[0 for col in range(num_nodes)] for row in range(num_nodes)]
start_node = random.choice(unused_node_list)
unused_node_list.remove(start_node)
terminal_node = random.choice(unused_node_list)
unused_node_list.remove(terminal_node)
path = []
path.append(start_node)
selected_node[start_node] = True

In [782]:
(start_node, terminal_node)

(1, 3)

In [783]:
path

[1]

In [784]:
selected_node[terminal_node]

False

In [785]:
while selected_node[terminal_node] == False:
    feasible_node = []
    for j in range(num_nodes):
        if A[start_node, j] > 0: # This node pair is connected        
            feasible_node.append(j)
    next_node = random.choice(feasible_node)
    path.append(next_node)
    
    # If there's a loop
    if selected_node[next_node]:
        path = path[:path.index(next_node)+1]
        start_node = next_node
        continue

    # If not, update a new initial node and add the edge to the graph
    selected_node[next_node] = True
    output[start_node][next_node] = 1
    output[next_node][start_node] = 1
    start_node = next_node

In [786]:
path

[1, 2, 4, 3]

In [787]:
selected_node

[False, True, True, True, True]

In [788]:
output

[[0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 1, 0]]

In [789]:
[i for i in range(len(selected_node)) if i == False]

[0]

In [790]:
# Continue picking a random remaining node and walk until we join the initial path

add_path = []
while (False in selected_node):
    remaining_node = [i for i in range(len(selected_node)) if i == False]
    new_start_node = random.choice(remaining_node)
    remaining_node.remove(new_start_node)
   
    if len(remaining_node) > 0:
        new_next_node = random.choice(remaining_node)
        new_feasible_node = []
        for j in range(num_nodes):
            if A[new_start_node, j] > 0: # This node pair is connected        
                new_feasible_node.append(j)
        new_next_node = random.choice(new_feasible_node)

    # Walk until it joins the original path
    if new_next_node in path:
        selected_node[new_next_node] = True
        add_path.append((new_start_node, new_next_node))
        output[new_start_node][new_next_node] = 1
        output[new_next_node][new_start_node] = 1
        break
    else: # not joining the original path yet: check for a loop
        if selected_node[new_next_node]:
            add_path = add_path[:add_path.index(next_node)+1]
            new_start_node = new_next_node
            continue
        else:
            selected_node[new_next_node] = True
            add_path.append((new_start_node, new_next_node))
            output[new_start_node][new_next_node] = 1
            output[new_next_node][new_start_node] = 1
            new_start_node = new_next_node

In [791]:
add_path

[(0, 1)]

In [792]:
output

[[0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 1, 0]]

In [793]:
for i in range(len(output)):
    for j in range(0+i, len(output)):
        if output[i][j] != 0:
            print("%d - %d: %d" % (i, j, output[i][j]))

0 - 1: 1
1 - 2: 1
2 - 4: 1
3 - 4: 1
